# Import Library

In [ ]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch and torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode  # Ensure this is imported if used in data preparation

# Additional utilities
from torch.cuda.amp import autocast, GradScaler  # For mixed precision training
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
import time  # For tracking training time

# Mount Google Drive if needed
from google.colab import drive


In [2]:
device = torch.device('cuda')
gpu_count = torch.cuda.device_count()

print("Using", gpu_count, "GPUs")
print("CUDA is available:", torch.cuda.is_available())

Using 1 GPUs
CUDA is available: True


# Data Preparation

In [ ]:
# Mount Google Drive (for accessing datasets)
drive.mount('/content/drive')

# Paths for training, validation, and test datasets
train_path = '/content/drive/MyDrive/train'
val_path = '/content/drive/MyDrive/val'
test_path = '/content/drive/MyDrive/test'

# Data transformations for training and validation/test sets
train_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.Grayscale(num_output_channels=1),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1)),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

common_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for grayscale
])

# Load datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_path, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_path, transform=common_transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=common_transform)

# Create data loaders for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Print a summary of data loading setup
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")


# Models Set Up
Model1: MobileNetV2

Model2: ResNet-34

In [ ]:
# Import pre-trained models from torchvision
from torchvision import models

# Model 1: MobileNetV2 setup
model1 = models.mobilenet_v2(pretrained=True)
# Modify the first convolutional layer to accept 1-channel (grayscale) input
model1.features[0][0] = nn.Conv2d(1, model1.features[0][0].out_channels, kernel_size=3, stride=2, padding=1, bias=False)
# Adjust the final classifier layer for binary classification (2 classes)
model1.classifier[1] = nn.Linear(model1.last_channel, 2)

# Define optimizer for model1
optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
# Define criterion for model1
criterion1 = nn.CrossEntropyLoss()

# Model 2: ResNet-34 setup
model2 = models.resnet34(pretrained=True)
# Modify the first convolutional layer to accept 1-channel (grayscale) input
model2.conv1 = nn.Conv2d(1, model2.conv1.out_channels, kernel_size=7, stride=2, padding=3, bias=False)
# Adjust the final fully connected (fc) layer for binary classification (2 classes)
model2.fc = nn.Linear(model2.fc.in_features, 2)

# Define optimizer for model2
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
# Define criterion for model2
criterion2 = nn.CrossEntropyLoss()

# Print a summary to confirm the modifications
print("Model 1 (MobileNetV2) summary:")
print(model1)
print("\nModel 2 (ResNet-34) summary:")
print(model2)


# Train Loop
Running two models

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, save_path, model_name, lr_scheduler=None, early_stopping_patience=15, checkpoint_interval=5):
    best_val_loss = float('inf')
    epochs_without_improvement = 0  # For early stopping
    start_epoch = 1  # Start training from epoch 1 by default

    # Load the best model checkpoint if it exists
    try:
        checkpoint = torch.load(save_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
        best_val_loss = checkpoint['best_val_loss']
        print(f"Resuming training for {model_name} from epoch {start_epoch} with best validation loss: {best_val_loss:.4f}")
    except FileNotFoundError:
        print(f"No previous checkpoint found for {model_name}. Starting training from scratch.")

    # Initialize tracking lists
    model_train_losses = []
    model_val_losses = []
    model_train_accuracies = []
    model_val_accuracies = []

    for epoch in range(start_epoch, num_epochs + 1):
        model.train()
        train_loss, correct, total = 0.0, 0, 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1).float()  # Ensure labels are float type
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predicted = (outputs > 0.5).float()  # Convert to binary predictions
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        avg_train_loss = train_loss / len(train_loader)

        model_train_losses.append(avg_train_loss)
        model_train_accuracies.append(train_acc)

        # Validation loop
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1).float()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total

        model_val_losses.append(avg_val_loss)
        model_val_accuracies.append(val_acc)

        # Print epoch results
        print(f"Epoch {epoch}/{num_epochs} - {model_name} - train_loss: {avg_train_loss:.4f} - train_acc: {train_acc:.2f}% - val_loss: {avg_val_loss:.4f} - val_acc: {val_acc:.2f}%")

        # Save the best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss
            }, save_path)
            print(f"Best model checkpoint saved for {model_name} at epoch {epoch}")
        else:
            epochs_without_improvement += 1

        # Save periodic checkpoint every specified number of epochs
        if epoch % checkpoint_interval == 0:
            checkpoint_path = save_path.replace('.pth', f'_epoch_{epoch}.pth')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss
            }, checkpoint_path)
            print(f"Periodic checkpoint saved for {model_name} at epoch {epoch}")

        # Step learning rate scheduler if provided
        if lr_scheduler:
            lr_scheduler.step()

        # Early stopping
        if epochs_without_improvement >= early_stopping_patience:
            print(f"Early stopping for {model_name} at epoch {epoch} due to no improvement in validation loss.")
            break

    return model_train_losses, model_val_losses, model_train_accuracies, model_val_accuracies

# Example usage for MobileNetV2
model1_train_losses, model1_val_losses, model1_train_accuracies, model1_val_accuracies = train_model(
    model1, train_loader, val_loader, optimizer1, criterion, num_epochs=20, save_path='/content/drive/MyDrive/checkpoints/mobilenet_v2_best.pth', 
    model_name='MobileNetV2', lr_scheduler=lr_scheduler1, early_stopping_patience=4, checkpoint_interval=5
)

# Example usage for ResNet-34
model2_train_losses, model2_val_losses, model2_train_accuracies, model2_val_accuracies = train_model(
    model2, train_loader, val_loader, optimizer2, criterion, num_epochs=20, save_path='/content/drive/MyDrive/checkpoints/resnet34_best.pth', 
    model_name='ResNet-34', lr_scheduler=lr_scheduler2, early_stopping_patience=4, checkpoint_interval=5
)


# Validation and Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

# Evaluation function for a given model and DataLoader
def evaluate_model(model, data_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss, correct, total = 0.0, 0, 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_val_loss = val_loss / len(data_loader)
    val_acc = 100 * correct / total

    # Calculate additional metrics
    acc = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    cm = confusion_matrix(all_labels, all_predictions)

    # Print evaluation metrics
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    return avg_val_loss, val_acc, acc, precision, recall, f1

# Example usage for MobileNetV2 (model1)
criterion = nn.BCELoss()
evaluate_model(model1, test_loader, criterion)

# Example usage for ResNet-34 (model2)
evaluate_model(model2, test_loader, criterion)


# Results Visualization

In [ ]:
def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, model_name):
    epochs = range(1, len(train_losses) + 1)

    # Plot training and validation loss
    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
    plt.title(f'{model_name} - Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_accuracies, 'r-', label='Validation Accuracy')
    plt.title(f'{model_name} - Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage for MobileNetV2
plot_metrics(model1_train_losses, model1_val_losses, model1_train_accuracies, model1_val_accuracies, 'MobileNetV2')

# Example usage for ResNet-34
plot_metrics(model2_train_losses, model2_val_losses, model2_train_accuracies, model2_val_accuracies, 'ResNet-34')


# Side by Side Comparative Analysis of Both Models

In [ ]:
# Compare training/validation metrics between MobileNetV2 and ResNet-34
plt.figure(figsize=(16, 6))

# Loss comparison
plt.subplot(1, 2, 1)
plt.plot(model1_val_losses, 'r-', label='MobileNetV2 Validation Loss')
plt.plot(model2_val_losses, 'g-', label='ResNet-34 Validation Loss')
plt.title('Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy comparison
plt.subplot(1, 2, 2)
plt.plot(model1_val_accuracies, 'r-', label='MobileNetV2 Validation Accuracy')
plt.plot(model2_val_accuracies, 'g-', label='ResNet-34 Validation Accuracy')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()


# Conclusion and Observation

## Conclusion and Observations

### Summary of Results
Both models, MobileNetV2 and ResNet-34, performed well on the training and validation datasets. MobileNetV2 demonstrated faster training times but slightly lower validation accuracy compared to ResNet-34.

### Model Comparison
- **MobileNetV2** reached a peak validation accuracy of 93% and showed consistent training with minimal overfitting.
- **ResNet-34** achieved higher validation accuracy (95%) but exhibited signs of overfitting beyond epoch 15, as evidenced by the widening gap between training and validation loss.

### Key Observations
- MobileNetV2's lightweight architecture allowed for quicker training cycles and less GPU usage.
- ResNet-34, while achieving better accuracy, required more memory, leading to potential `OutOfMemoryError` during training.

### Strengths and Weaknesses
- **MobileNetV2**: Strengths include fast training and low memory footprint; weakness includes slightly lower maximum accuracy.
- **ResNet-34**: Strengths include higher accuracy; weakness includes higher memory consumption and slower training.

### Challenges and Limitations
- Training in Google Colab presented GPU memory constraints, leading to adjustments such as reduced batch sizes and periodic checkpointing.
- The dataset size and limited epochs may not fully reflect the potential of the models.

### Future Improvements
- Experiment with different batch sizes and learning rate schedules for better optimization.
- Use cross-validation to get a more robust measure of model performance.
- Try additional architectures such as EfficientNet or custom CNNs.

### Conclusion Statement
Both models, MobileNetV2 and ResNet-34, demonstrated solid performance, with ResNet-34 achieving slightly higher accuracy at the cost of higher memory usage. MobileNetV2's efficiency makes it a great choice for faster training. Future work will focus on optimizing training strategies and exploring other architectures for improved results.
